<a href="https://colab.research.google.com/github/jdmakubex/diccionarios/blob/main/Modelo_Bert_en_Espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Construcción de un modelo BERT en español para clasificación multiclase con una sola salida**

Como primer paso, se instalan las librerías necesarias


In [11]:
!pip install simpletransformers
!pip freeze | grep simpletransformers

     |████████████████████████████████| 231 kB 4.9 MB/s 
     |████████████████████████████████| 8.3 MB 15.1 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 1.7 MB 26.3 MB/s 
     |████████████████████████████████| 3.3 MB 36.6 MB/s 
     |████████████████████████████████| 290 kB 37.3 MB/s 
     |████████████████████████████████| 3.1 MB 36.3 MB/s 
     |████████████████████████████████| 1.2 MB 38.9 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 895 kB 44.1 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 180 kB 50.5 MB/s 
     |████████████████████████████████| 139 kB 46.2 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 243 kB 47.1 MB/s 
     |████████████████████████████████| 1.1 MB 34.2 MB/s 
     |█████████████████

simpletransformers==0.62.4


Verificamos la correcta instalación del paquete
 simpletransformer


In [12]:
!pip freeze | grep simpletransformers

simpletransformers==0.62.4


Como siguiente paso se importan los datos de entrenamiento para el modelo multiclase, se proponen 12 clases de clasificación:


In [13]:
!wget https://jdmakubex.github.io/diccionarios/diccionarios/trainGold.csv
!wget https://jdmakubex.github.io/diccionarios/diccionarios/test.csv


--2021-11-03 05:41:09--  https://jdmakubex.github.io/diccionarios/diccionarios/trainGold.csv
Resolving jdmakubex.github.io (jdmakubex.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jdmakubex.github.io (jdmakubex.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 612853 (598K) [text/csv]
Saving to: ‘trainGold.csv.2’

trainGold.csv.2     100%[===================>] 598.49K  --.-KB/s    in 0.04s   

2021-11-03 05:41:09 (13.8 MB/s) - ‘trainGold.csv.2’ saved [612853/612853]

--2021-11-03 05:41:09--  https://jdmakubex.github.io/diccionarios/diccionarios/test.csv
Resolving jdmakubex.github.io (jdmakubex.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jdmakubex.github.io (jdmakubex.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13535 (13K) [text/csv]
Saving to: ‘test.csv.2’

test.csv.2          100%[===================>]  

A continuación se importa la librería pandas, y se instancia como pd, pandas nos sirve para manipular los datos importados en forma de dataFrames.


In [14]:
import pandas as pd

In [15]:
class_list = ['Accidentes e Incidentes','Arte y Cultura','Bloqueos','Neutral','Delito','Deportes','Desastres naturales','Economía','Manifestaciones','Política','Religión','Salud']

Leemos el archivo CSV trainGold, el cual ya fué previamente importado, y lo cargamos como dataframe

In [16]:
df = pd.read_csv('trainGold.csv',sep=',',encoding='utf8',names=["texto", "tipo"])

Mostramos el contenido del dataframe, verificamos las columnas, texto, y tipo

In [17]:
df

,texto,tipo
0,Solicitarán apoyo para 3 mil afectados por fue...,Accidentes e Incidentes
1,Un muerto y daños materiales en accidente sobr...,Accidentes e Incidentes
2,Alerta Protección civil de probables inundacio...,Desastres naturales
3,Protección Civil mantiene monitoreo de niveles...,Accidentes e Incidentes
4,Extorsión mediante apps de préstamo; la nueva ...,Accidentes e Incidentes
...,...,...
6115,Ser volovanero y sobrevivir a la pandemia…,Arte y Cultura
6116,Acorrala COVID-19 a Ciudad Ixtepec se reportan...,Arte y Cultura
6117,Aumentan contagios de COVID-19 en jóvenes de 1...,Arte y Cultura
6118,Recibe juez amparo de clínica del ISSSTE se di...,Arte y Cultura


Aplicando lambda x, todo el  contenido de la columna tipo, la pasamos a entero, para efecto del correcto procesamiento de los algoritmos, esa transformación la guardamos en una nueva columna denominada pred_class

In [18]:
df['pred_class'] = df.apply(lambda x:  class_list.index(x['tipo']),axis=1)

reconstruimos el dataframe, esta vez, dejando solo las columnos que vamos utilizar: texto y pred_class
también, mostramos cuantos registros tenemos en el dataframe y cuantas columnas contiene. Además, mostramos el dataframe resultante ya con los datos procesados.

In [19]:
df = df[['texto','pred_class']]
print(df.shape)
df.head()

(6120, 2)


,texto,pred_class
0,Solicitarán apoyo para 3 mil afectados por fue...,0
1,Un muerto y daños materiales en accidente sobr...,0
2,Alerta Protección civil de probables inundacio...,6
3,Protección Civil mantiene monitoreo de niveles...,0
4,Extorsión mediante apps de préstamo; la nueva ...,0


A continuación se importa el módulo train_test_split, con el que nos apoyamos para extraer una muestra de los datos de entrenamoento, en este caso se definió un tamaño del 20% de los datos (2 columnas con 1224 filas), los cuales tomarno para la variable test, y el otro 80% de los datos (2 columnas con 4896 filas) quedó en la variable de entrenamiento.

In [20]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (4896, 2)
test shape:  (1224, 2)


Con la librería simpletransformers se importa un modelo preentrenado en español, este modelo está utilizando 16 neuronas, las cuales se entrenan en 6 épocas, además se define que el entrenamiento es pare 12 etiquetas de clasificación. y es importante recalcar que el modelo utilizado es bert, en su versión en español, instanciado con: "bert", "dccuchile/bert-base-spanish-wwm-cased"

In [21]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 6}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "dccuchile/bert-base-spanish-wwm-cased",
    num_labels=12,
    args=train_args
)

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/469k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Como siguiente paso, se inicia el entrenamiento del modelo, y es en esta parte donde se entrenan las neuronas del modelo, con 6 épocas de entrenamiento se tuvo una acertividad del 76%, se puede optimar màs, pero requiere mas capacidad de còmputo y tiempo de ejecusión, pero es aceptable como primera prueba. Como entrenamiento inicial pasa la prueba.

In [22]:
# Entrenamiento del Modelo
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4896 [00:00<?, ?it/s]

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/612 [00:00<?, ?it/s]

Running Epoch 1 of 6:   0%|          | 0/612 [00:00<?, ?it/s]

Running Epoch 2 of 6:   0%|          | 0/612 [00:00<?, ?it/s]

Running Epoch 3 of 6:   0%|          | 0/612 [00:00<?, ?it/s]

Running Epoch 4 of 6:   0%|          | 0/612 [00:00<?, ?it/s]

Running Epoch 5 of 6:   0%|          | 0/612 [00:00<?, ?it/s]

(3672, 0.766319060346623)

A continuación se imprimen los parametros del modelo, para analizar, finalmente, el modelo tiene un grado de acertividad de predicción 65% de esto también es en parte a la calidad de la muestrea de los datos. Debido a que la muestra se produjo de forma automatizada, no supervizada. Lo cual hace un poco mas grande el cesgo.

In [23]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result



/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1224 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/153 [00:00<?, ?it/s]

{'acc': 0.6519607843137255,
 'eval_loss': 1.1594209118314038,
 'f1': 0.6519607843137255,
 'mcc': 0.6074205927220314}

Se recopila el modelo, y se guarda, para no tener que ejecutar el entrenamiento del modelo nuevamente, con esto se pretende ahorrar tiempo y cómputo del modelo.

In [24]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

El modelo, para este proyecto se denominó: jdmodel-ucuahutemoc-spanish, y se guarda como un archivo Tar GZ

In [25]:
save_model('outputs','jdmodel-ucuahutemoc-spanish')

In [26]:
!tar -zxvf ./jdmodel-ucuahutemoc-spanish.tar.gz

outputs/model_args.json
outputs/eval_results.txt
outputs/pytorch_model.bin
outputs/tokenizer_config.json
outputs/tokenizer.json
outputs/training_args.bin
outputs/config.json
outputs/special_tokens_map.json
outputs/vocab.txt


Se borran los archivos de salida del entreganmiento del modelo.

In [27]:
!rm -rf outputs

Para verificar el funcionamiento del modelo, se hace una prueba, ejecutando la Carga del Modelo y realizando un ejemplo de su uso. Se descomprime el modelo.

In [28]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('jdmodel-ucuahutemoc-spanish')

Se definen nuevamente los parametros del modelo

In [29]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 6}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "outputs/",
    num_labels=12,
    args=train_args
)

Se declara una lista de etiqueda, y se dimula en la variable test_tweet, un texto como sería un tweet al ser capturado en el streaming. Se manda la variable al método predictions, y para este caso, el resultado fué delito, lo cual concuerda, con el texto por que se mencionan aspectos como portación de armas, y drogas, los cuales se consideran deltos.

In [30]:
class_list = ['Accidentes e Incidentes','Arte y Cultura','Bloqueos','Neutral','Delito','Deportes','Desastres naturales','Economía','Manifestaciones','Política','Religión','Salud']

test_tweet = "Fiscalía General del Estado GEO detiene a dos personas por portar armas y drogas"

predictions, raw_outputs = model.predict([test_tweet])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Delito


In [31]:
%ls


cache_dir/                          sample_data/  trainGold.csv
jdmodel-ucuahutemoc-spanish.tar.gz  test.csv      trainGold.csv.1
outputs/                            test.csv.1    trainGold.csv.2
runs/                               test.csv.2
